# Validation of the PyPSA-Africa solar and wind potential

This notebooks investigates the data quality of the African solar and wind potential
data by comparing PyPSA, and other research results. There are two types of **potentials**, 
- technical installable power potential [MW] and 
- technical 'resource' energy potential [MWh]
This notebook looks only at the technical resource energy potential that was sometimes cited in literature. The `build_renewable_profile.ipynb` offers plots on the installable resource potential.

To reproduce the findings obtained in this notebook,
please run the full snakemake workflow for the Africa.
To do so, please set ``countries = ["Africa"]`` in the ``config.yaml`` file.

Note. An unoptimized prepared network is sufficient for this notebook. 

#### Generate profiles (if not already available)
If you don't have the renewable profiles such as:
- `resources/profile_solar.nc`
- `resources/profile_onwind.nc`
- `resources/profile_offwind-ac.nc`

you could try to generate them from the terminal. In this case set the path so that you are located at ~/pypsa-africa. Once you are there, use the command:

```
snakemake -j 1 resources/profile_{technology}.nc
```

replace {technology} for the timeseries of interest.

## Preparation

### Import packages

In [1]:
import atlite
import cartopy.crs as ccrs
import xarray as xr
import geopandas as gpd

# import holoviews as hv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import pypsa
import shutil
from rasterio.plot import show
from atlite.gis import shape_availability, ExclusionContainer


plt.rcParams["figure.figsize"] = [7, 7]
%matplotlib inline

### Set main directory to root folder

In [2]:
# set current folders
import sys

sys.path.append("../../")  # adds path to $ .../pypsa-africa
from scripts._helpers import sets_path_to_root

sets_path_to_root("pypsa-africa")  # moves path to root

This is the repository path:  /home/max/OneDrive/PHD-Flexibility/07_pypsa-africa/0github/pypsa-africa/pypsa-africa
Had to go 2 folder(s) up.


## 1. Load and open the PyPSA-Africa generated profiles
The `build_renewable_profiles` for each technology are stored in the `resources` folder. We read first the path and open then with xarray the .nc file. As you can see there is quite a lot data available.

In [3]:
solar_path = os.path.realpath("resources") + "/profile_solar.nc"
solar = xr.open_dataset(solar_path)
solar

wind_path = os.path.realpath("resources") + "/profile_onwind.nc"
wind = xr.open_dataset(wind_path)
wind

<xarray.Dataset>
Dimensions:           (time: 8760, bus: 2848, y: 257, x: 291)
Coordinates:
  * time              (time) datetime64[ns] 2013-01-01 ... 2013-12-31T23:00:00
  * bus               (bus) object '29' '707' '708' ... '7036' '7038' '7039'
  * y                 (y) float64 -37.5 -37.2 -36.9 -36.6 ... 38.7 39.0 39.3
  * x                 (x) float64 -19.5 -19.2 -18.9 -18.6 ... 66.9 67.2 67.5
Data variables:
    profile           (time, bus) float64 ...
    weight            (bus) float64 ...
    p_nom_max         (bus) float64 ...
    potential         (y, x) float64 ...
    average_distance  (bus) float64 ...

## 2. Calculate PyPSA-Africa technical potentials
- `profiles` are hourly profiles and given as `per unit` values for each bus (region)
- `per-unit` is scaled according to the `p_nom_max` of each bus (region)
- To calculate the potential across Africa we need to do the following:

```p_nom_max * profiles and sum the values for each region and timestep```

In [4]:
technical_solar_potential_twh = (
    sum(solar["p_nom_max"] * solar["profile"]).sum() / 10**6
)  # MWh to TWH conversion
technical_wind_potential_twh = (
    sum(wind["p_nom_max"] * wind["profile"]).sum() / 10**6
)  # MWh to TWH conversion

print(
    "Socio-technical solar potential Africa with exclusion area - PyPSA 2022:",
    technical_solar_potential_twh.values.round(),
    "TWh",
)
print(
    "Socio-technical wind potential Africa with exclusion area - PyPSA 2022:",
    technical_wind_potential_twh.values.round(),
    "TWh",
)

print(
    "Comparing with the below values. Wind seems ok and solar seems low. Important is that PyPSA is mentioning the socio-technical wind potential which is assumed to be between 5-10% of the technical potential. More discussion are given here: https://github.com/pypsa-meets-africa/pypsa-africa/issues/361"
)

Socio-technical solar potential Africa with exclusion area - PyPSA 2022: 45166.0 TWh
Socio-technical wind potential Africa with exclusion area - PyPSA 2022: 108703.0 TWh
Comparing with the below values. Wind seems ok and solar seems low. Important is that PyPSA is mentioning the socio-technical wind potential which is assumed to be between 5-10% of the technical potential. More discussion are given here: https://github.com/pypsa-meets-africa/pypsa-africa/issues/361


Comparing this to other sources.

WIND:
- International Finance Cooperation reports 180.000 TWh technical wind potential for cut-in speed of 6.0 m/s (10.1016/j.renene.2021.07.021)
- International Finance Cooperation reports 118.700 TWh technical wind potential for cut-in speed of 7.5 m/s (10.1016/j.renene.2021.07.021)
- International Finance Cooperation reports 64.700 TWh technical wind potential for cut-in speed of 8.5 m/s (10.1016/j.renene.2021.07.021)
- Mentis et al. (2015) reports 34.000 TWh technical wind potential (10.1016/j.renene.2015.03.072)
- IRENA 2014 reports 66.600 TWh technical potential for turbines with capacity factors above >30%(Title: Estimating the Renewable Energy Potential in Africa. A GIS-based approach) https://www.irena.org/publications/2014/Aug/Estimating-the-Renewable-Energy-Potential-in-Africa-A-GIS-based-approach

SOLAR:
- IRENA 2014 reports 660.000 TWh technical solar PV potential (Title: Estimating the Renewable Energy Potential in Africa. A GIS-based approach) https://www.irena.org/publications/2014/Aug/Estimating-the-Renewable-Energy-Potential-in-Africa-A-GIS-based-approach

## 3. Wikipedia list of largest PV plant

In case the above data is off, we might want to look at the "installable power density" [MW/km2]. The installable power density describes how much of the technology can be build in a specific area while being maintainable and techno-economic efficient (reduce shaddow losses and alike). This parameter is an input in too energy system models that finally lead to the technical potential [TW].

In [5]:
wikipedia_link = "https://en.m.wikipedia.org/wiki/List_of_photovoltaic_power_stations"
wikipedia_pv_list = pd.read_html(wikipedia_link)
df = pd.DataFrame(wikipedia_pv_list[1])
# df

In [6]:
# tuning and cleaning
df = df.rename(
    columns={"CapacityMWDC or MWAC\xa0(*)": "p_nom", "LandSize(km2)": "area"}
)
df.loc[:, "area"] = df.loc[:, "area"].str.extract("(\d+)", expand=False)
df.loc[:, "p_nom"] = df.loc[:, "p_nom"].str.extract("(\d+)", expand=False)
df1 = df.copy()
df1.dropna(inplace=True, subset=["area", "p_nom"])
df1.loc[:, "area"] = df1["area"].astype(float)
df1.loc[:, "p_nom"] = df1["p_nom"].astype(float)

# manual cleaning
df1.loc[59, "area"] = 2.6  #  hand checked
df1 = df1.drop(2)

df1["MW/km2"] = df1["p_nom"] / df1["area"]
print("Average installed power density", df1["MW/km2"].mean())
print("Stats [MW/km2]:", df1["MW/km2"].describe())
print("Stats [Year:", df1["Year"].astype("int").describe())

Average installed power density 45.62385307011756
Stats [MW/km2]: count     41.000000
mean      45.623853
std       30.368618
min       10.416667
25%       28.947368
50%       38.679245
75%       49.400000
max      147.125000
Name: MW/km2, dtype: float64
Stats [Year: count      41.000000
mean     2017.390244
std         2.322478
min      2012.000000
25%      2016.000000
50%      2018.000000
75%      2019.000000
max      2021.000000
Name: Year, dtype: float64


## 4. Comparison to other sources

In [7]:
africa_km2 = 30 * 10**6
pv_panel_power_density = 0.2  # kWp/m2  https://en.wikipedia.org/wiki/Solar_panel
pv_efficiency = 0.15
capacity_factor = 0.2
year2hourly = 8700
kwh2twh = 10 ** (-9)
mwh2twh = 10 ** (-6)
kw2mw = 10 ** (-3)
m2tokm2 = 10 ** (-6)

# Benban
p_nom = 1650  # MW
field_area = 37  # km2
specific_power_benban = p_nom / field_area  # MW/km2
technical_potential_benban = p_nom * mwh2twh * year2hourly * capacity_factor  # TWh


# IRENA
# Equation: Solar Resource Availability x PV Module Efficiency / Spacing Factor = specific Technical Potential [TWh/km2]
solar_availability = 2000  # [kWh/m2/year]
pv_efficiency_irena = 0.165  # [%]
spacing_factor = 5  #
installable_power_density_irena = (
    solar_availability / spacing_factor / year2hourly * kw2mw / m2tokm2
)

# Pietzcker 2014
specific_technical_potential_high_pietz = 1 / 0.009  # [1/km2/MW]
specific_technical_potential_low_pietz = 1 / 0.016  # [1/km2/MW]

# Calliope https://github.com/calliope-project/euro-calliope/blob/9f142903f543dcb0634be7fff8b07f33305811ed/config/default.yaml#L40
installable_power_density_calliope = 80

# back-of-the-envelope calculation
installable_power_density_envelope = (
    pv_panel_power_density / spacing_factor / m2tokm2 * kw2mw
)
technical_potential_envelope = (
    installable_power_density_envelope
    * africa_km2
    * year2hourly
    * mwh2twh
    * capacity_factor
)

# PyPSA-Eur
max_installable_power = 40  # [MW/km2]
correction_factor = 0.05  #  If 100% are installable only 5% can be used
specific_technical_potential = max_installable_power * correction_factor
installable_power_density_pypsa = 1.7
technical_potential_pypsa = (
    installable_power_density_pypsa
    * africa_km2
    * year2hourly
    * capacity_factor
    * mwh2twh
)


# Specific Technical potential [MW/km2]
print("Investigating config.yaml input for PyPSA-Africa:")
print(
    "Installed power density, Benban plant Egypt:",
    round(specific_power_benban, 2),
    "MW/km2",
)
print(
    "Installable power density, IRENA:",
    round(installable_power_density_irena),
    "MW/km2",
)
# Fraunhofer 2022 https://www.ise.fraunhofer.de/content/dam/ise/de/documents/publications/studies/aktuelle-fakten-zur-photovoltaik-in-deutschland.pdf
print(
    "Installable power density, Fraunhofer 2022:",
    round(1, 2),
    "MW/ha which equals",
    round(1 / 0.01, 2),
    "MW/km2",
)  # p.40
print(
    "Installable power density, Fraunhofer 2022 old assumption (2014):",
    round(0.35, 2),
    "MW/ha which equals",
    round(0.35 / 0.01, 2),
    "MW/km2",
)  # p.41
print(
    "Installable power density, Pietzer 2014 - high:",
    round(specific_technical_potential_high_pietz),
    "MW/km2",
)
print(
    "Installable power density, Pietzer 2014 - low:",
    round(specific_technical_potential_low_pietz),
    "MW/km2",
)
print(
    "Installable power density - Calliope 2022:",
    round(installable_power_density_calliope, 2),
    "MW/km2",
)
print(
    "Installable power density, Back-of-the-envelope calculation:",
    round(installable_power_density_envelope),
    "MW/km2",
)
print(
    "Installable power density - PyPSA 2022:",
    round(installable_power_density_pypsa, 2),
    "MW/km2",
)
print(
    "Installable power density - New PyPSA 2022:",
    round(4.6, 2),
    "MW/km2.  # Based on discussion in https://github.com/pypsa-meets-africa/pypsa-africa/issues/361",
)

# Specific Technical potential [TWh/km2]
print("")
print("Solar. Outputs:")
print(
    "Specific technical potential, Back-of-the-envelope:",
    round(technical_potential_envelope / africa_km2, 2),
    "TWh/km2",
)
print(
    "Specific technical potential, oberved in Benban:",
    round(technical_potential_benban / field_area, 3),
    "TWh/km2",
)
print(
    "Specific socio-technical potential without exclusion area, PyPSA:",
    round(technical_potential_pypsa / africa_km2, 3),
    "TWh/km2",
)
print(
    "Specific socio-technical potential with exclusion area, PyPSA:",
    (technical_solar_potential_twh / africa_km2).values.round(3),
    "TWh/km2",
)
print(
    "Specific socio-technical potential with exclusion area, New PyPSA 2022:",
    (technical_solar_potential_twh * (4.6 / 1.7) / africa_km2).values.round(3),
    "TWh/km2.  # Driver is the installable power density",
)

# Technical potential [TWh]
print("")
print(
    "Technical potential Africa, Back-of-the-envelope calculation without exclusion areas:",
    round(technical_potential_envelope),
    "TWh",
)
print("Technical potential with exclusion areas - IRENA: 660000 TWh")
print(
    "Socio-technical potential Africa without exclusion area - PyPSA 2022:",
    round(technical_potential_pypsa),
    "TWh",
)
print(
    "Socio-technical potential Africa with exclusion area - PyPSA 2022:",
    technical_solar_potential_twh.values.round(),
    "TWh",
)
print(
    "Socio-technical potential Africa with exclusion area - New PyPSA 2022:",
    (technical_solar_potential_twh * (4.6 / 1.7)).values.round(),
    "TWh.  # Driver is the installable power density",
)
print(
    "Technical potential, observed in Benban:", round(technical_potential_benban), "TWh"
)

print("")
print("Wind. Outputs:")
print(
    "Socio-technical potential Africa with exclusion area - PyPSA 2022:",
    technical_wind_potential_twh.values.round(),
    "TWh",
)

Investigating config.yaml input for PyPSA-Africa:
Installed power density, Benban plant Egypt: 44.59 MW/km2
Installable power density, IRENA: 46 MW/km2
Installable power density, Fraunhofer 2022: 1 MW/ha which equals 100.0 MW/km2
Installable power density, Fraunhofer 2022 old assumption (2014): 0.35 MW/ha which equals 35.0 MW/km2
Installable power density, Pietzer 2014 - high: 111 MW/km2
Installable power density, Pietzer 2014 - low: 62 MW/km2
Installable power density - Calliope 2022: 80 MW/km2
Installable power density, Back-of-the-envelope calculation: 40 MW/km2
Installable power density - PyPSA 2022: 1.7 MW/km2
Installable power density - New PyPSA 2022: 4.6 MW/km2.  # Based on discussion in https://github.com/pypsa-meets-africa/pypsa-africa/issues/361

Solar. Outputs:
Specific technical potential, Back-of-the-envelope: 0.07 TWh/km2
Specific technical potential, oberved in Benban: 0.078 TWh/km2
Specific socio-technical potential without exclusion area, PyPSA: 0.003 TWh/km2
Specific